In [1]:
import pandas as pd
import numpy as np
import os
import tqdm
import shutil

In [5]:
dataset = 'session_emma' # onion or emma
results_dir = f'../results/' + dataset + '/performance/'
#cutoffs = [1, 5, 10, 20]
cutoffs = [5]
cutoff_date = '2025_03_25'
metrics = ['Recall', 'Precision', 'nDCG', 'MRR', 'HR', 'EFD', 'EPC', 'ARP', 'PopREO', 'PopRSP', 'ItemCoverage', 'UserCoverage']
#metrics = ['Recall', 'Precision', 'nDCG', 'MRR', 'HR']

In [6]:
def find_modality(string):
    keyword = "modalites="
    start = string.find(keyword)
    if start == -1:
        return None

    start += len(keyword)
    end = string.find("_", start)
    if end == -1:
        end = len(string)

    modality = string[start:end].strip("'")
    return modality

In [7]:
# read all files in result folder
files = os.listdir(results_dir)
df = pd.DataFrame()

for cutoff in cutoffs:
    df_cut = pd.DataFrame()
    result_files_cutoff = [f for f in files if f.startswith(f'rec_cutoff_{cutoff}_')]
    for f in result_files_cutoff:
        df_tmp = pd.read_csv(results_dir + f, sep='\t')
        tmp_metrics = [m for m in metrics if m in df_tmp.columns]
        df_tmp.rename(columns={m: f'{m}@{cutoff}' for m in tmp_metrics}, inplace=True)
        df_tmp['modality'] = find_modality(df_tmp['model'].values[0])
        df_tmp['path'] = df_tmp['model']
        df_tmp['model'] = df_tmp['model'].str.split('_').str[0]
        df_tmp['date'] = f[-23:-4]
        df_cut = pd.concat([df_cut, df_tmp], axis=0)

    # keep only newest model results
    df_cut = df_cut.sort_values('date').drop_duplicates(subset=['model', 'modality'], keep='last')

    # drop models results before cutoff date
    df_cut = df_cut[df_cut['date'] >= cutoff_date]

    # merge with previous cutoffs
    if not df.empty:
        df = pd.merge(df, df_cut, on=['model', 'date', 'modality'], how='inner')
        print(df.shape)
    else:
        df = df_cut

df.sort_values('nDCG@5', inplace=True)
# display columns with @10 only
df[['model', 'modality', 'date', 'path'] + [c for c in df.columns if '@' in c]]
#df[['model', 'modality', 'date'] + ['Recall@1', 'Recall@5', 'Recall@10', 'nDCG@1', 'nDCG@5', 'nDCG@10']]

,model,modality,date,path,Recall@5,Precision@5,nDCG@5,MRR@5,HR@5
0,LightGCNM,emotion,2025_04_05_17_55_19,LightGCNM_seed=123_e=200_bs=128_lr=0$001_facto...,0.194679,0.063761,0.163467,0.192842,0.294566
0,LightGCNM,audio'-'textual'-'visual,2025_04_05_17_54_31,LightGCNM_seed=123_e=200_bs=128_lr=0$001_facto...,0.200758,0.066036,0.167624,0.198208,0.304848
0,LightGCNM,audio'-'textual'-'visual'-'emotion,2025_04_05_18_29_03,LightGCNM_seed=123_e=200_bs=128_lr=0$001_facto...,0.200022,0.066117,0.168188,0.199721,0.306006
0,FeatureItemKNN,emotion,2025_04_05_17_12_23,FeatureItemKNN_nn=100_sim=cosine_msf=0$05_bin=...,0.201069,0.066213,0.169153,0.201116,0.303554
0,LightGCN,None,2025_04_05_17_19_37,LightGCN_seed=123_e=200_bs=128_lr=0$0005_facto...,0.204364,0.067452,0.170075,0.201043,0.310568
0,FeatureItemKNN,audio'-'textual'-'visual'-'emotion,2025_04_05_17_28_11,FeatureItemKNN_nn=100_sim=cosine_msf=0$05_bin=...,0.206836,0.067847,0.173227,0.205704,0.311045
0,ItemKNN,None,2025_04_05_16_45_08,ItemKNN_nn=100_sim=cosine_imp=standard_bin=Fal...,0.207649,0.068078,0.173430,0.205419,0.311725
0,FeatureItemKNN,audio'-'textual'-'visual,2025_04_05_17_13_37,FeatureItemKNN_nn=70_sim=cosine_msf=0$05_bin=F...,0.207557,0.068092,0.173467,0.205762,0.311862


In [8]:
for model_path in df['path'].values:
    print(model_path)

LightGCNM_seed=123_e=200_bs=128_lr=0$001_factors=64_l_w=1e-05_modalites='emotion'_n_layers=2_normalize=True_aggregation=concat_loads='EmotionAttribute'
LightGCNM_seed=123_e=200_bs=128_lr=0$001_factors=64_l_w=1e-05_modalites='audio'-'textual'-'visual'_n_layers=2_normalize=True_aggregation=concat_loads='AudioAttribute'-'TextualAttribute'-'VisualAttribute'
LightGCNM_seed=123_e=200_bs=128_lr=0$001_factors=64_l_w=1e-05_modalites='audio'-'textual'-'visual'-'emotion'_n_layers=2_normalize=True_aggregation=concat_loads='AudioAttribute'-'TextualAttribute'-'VisualAttribute'-'EmotionAttribute'
FeatureItemKNN_nn=100_sim=cosine_msf=0$05_bin=False_modalites='emotion'_aggregation=ensemble_loads='EmotionAttribute'
LightGCN_seed=123_e=200_bs=128_lr=0$0005_factors=64_l_w=1e-05_n_layers=3_normalize=True
FeatureItemKNN_nn=100_sim=cosine_msf=0$05_bin=False_modalites='audio'-'textual'-'visual'-'emotion'_aggregation=concat_loads='AudioAttribute'-'TextualAttribute'-'VisualAttribute'-'EmotionAttribute'
ItemKNN_

In [13]:
# export to comma seperated file
df.to_csv(f'../results/{dataset}_performance.csv', index=False)

In [16]:
for cutoff in cutoffs:
    df_cut = df[['model'] + [c for c in df.columns if f'@{cutoff}' in c]]
    df_cut.to_csv(f'../results/{dataset}_performance_{cutoff}.csv', index=False)